In [63]:
import os

In [64]:
os.getcwd()

'c:\\Users\\mwori\\Deep-Learning-Project'

In [66]:
%pwd

'c:\\Users\\mwori\\Deep-Learning-Project'

In [67]:
from  dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path
    

In [68]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [69]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )
        return prepare_callback_config
    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chicken-fecal-images")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path= Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config


    

In [70]:
import time

In [71]:
class PrepareCallback:
    def __init__(self, config:PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}"
        )
        # save the Model's Metrics like accuracy and loss
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

    @property
    def _create_ckpt_callbacks(self):
        #saving the model during or after training when it improves performance
        #lowest validation loss or highest validation accuracy
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=str(self.config.checkpoint_model_filepath),
            save_best_only=True
        )
    
    def get_tb_ckpt_callbacks(self):
        return[
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]
#2:15:57
        

In [72]:
import os 
import urllib.request as request
from zipfile import ZipFile
import  tensorflow as tf
import time


In [73]:
class Training:
    def __init__(self, config:TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):
        # responsible for Image Augmetation
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.20
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            #resizing images by averageing pixel values
            interpolation = "bilinear"
        )
        # Augmenting generated Images
        valid_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_data_generator.flow_from_directory(
            directory = self.config.training_data,
            subset="validation",
            shuffle = False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range = 40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_data_generator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = "training",
            shuffle=True,
            **dataflow_kwargs
        )
    

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self, callback_list: list):
        #Number of batches the model will take during training
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        #Number of batches the model will take during validation
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size


        self.model.fit(
            self.train_generator,
            epochs = self.config.params_epochs,
            steps_per_epoch = self.steps_per_epoch,
            validation_steps = self.validation_steps,
            validation_data = self.valid_generator,
            callbacks = callback_list
        )


        self.save_model(
            path = self.config.trained_model_path,
            model=self.model
        )



In [75]:
try:

    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list= callback_list
    )
except Exception as e:
    raise e


[2024-08-14 20:30:24,816: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-14 20:30:25,007: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-14 20:30:25,074: INFO: common: created directory at: artifacts]
[2024-08-14 20:30:25,136: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2024-08-14 20:30:25,189: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
[2024-08-14 20:30:26,693: INFO: common: created directory at: artifacts\training]
Found 78 images belonging to 2 classes.
Found 312 images belonging to 2 classes.
Epoch 1/10
19/19 [==============================] - 108s 6s/step - loss: 12.4327 - accuracy: 0.5304 - val_loss: 11.4808 - val_accuracy: 0.3906
Epoch 2/10
19/19 [==============================] - 103s 5s/step - loss: 5.4609 - accuracy: 0.7500 - val_loss: 10.6558 - val_accuracy: 0.3906
Epoch 3/10
19/19 [==============================] - 112s 6s/step - loss: 5.4254 - accur